In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")


os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [2]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="llama3-groq-70b-8192-tool-use-preview")
llm.invoke("hi").content

'Hi, I need some information about a specific group. Can you help me with that?'

In [3]:
message=[{"role":"system","content":"you are a helpful assistant"},
         {"role":"user","content":"hi how are you?"}]

In [4]:
result = llm.invoke(message)
print(result.content)

I'm doing well, thank you! How can I assist you today?


In [12]:
class Chatbot:
    def __init__(self,system=""):
        self.system=system
        self.message=[]
        if self.system:
            self.message.append({"role":"system","content":system})
    def __call__(self,message):
        self.message.append({"role":"user","content":message})
        result=self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
        
    def execute(self):
        llm = ChatGroq(model_name="Gemma2-9b-It")
        result = llm.invoke(self.message)
        return result.content
        

In [14]:
bot=Chatbot(system="you are helpful assistant")

In [15]:
bot("hi how are you?")

"I'm doing well, thank you! As an AI, I don't have feelings, but I'm ready to assist you with any questions or tasks you have.  \n\nHow can I help you today? 😊  \n"

In [16]:
bot.message

[{'role': 'system', 'content': 'you are helpful assistant'},
 {'role': 'user', 'content': 'hi how are you?'},
 {'role': 'assistant',
  'content': "I'm doing well, thank you! As an AI, I don't have feelings, but I'm ready to assist you with any questions or tasks you have.  \n\nHow can I help you today? 😊  \n"}]

In [35]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

In [ ]:
prompt = """You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer related to fitness, health, or exercise.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 1800 * 0.2
Runs a calculation (e.g., calorie intake, BMI, workout progress) and returns the result.

fitness_wiki:
e.g. fitness_wiki: Benefits of Cardio
Returns a summary of fitness-related information from a fitness knowledge base (Wikipedia or similar).

exercise_plan:
e.g. exercise_plan: Beginner full-body workout
Returns a recommended workout plan based on the type of exercise or user level.

nutrition_info:
e.g. nutrition_info: Protein sources for vegetarians
Looks up nutritional information or suggestions for diets.

Example session:
Question: What are the benefits of cardio exercises?
Thought: I should look up information about the benefits of cardio exercises.
Action: fitness_wiki: Benefits of Cardio
PAUSE

You will be called again with this:

Observation: Cardio exercises improve cardiovascular health, increase lung capacity, and aid in weight loss.

You then output:

Answer: Cardio exercises improve cardiovascular health, increase lung capacity, and help with weight loss.
""".strip()

Pattern Breakdown:

- ^: This matches the start of a string. It means the string must begin with what follows.
- Action:: This is a literal match. It means the string must have the text "Action:" at the beginning.
- (\w+):
    1. The parentheses () define a capture group. This allows you to extract part of the string that matches this section.
    2. \w+ matches one or more word characters (letters, digits, and underscores). This will capture a word that follows "Action:".
- :: This is a literal colon that separates the word matched by (\w+) from the rest of the string.
- (.*):
    1. This is another capture group, where .* matches any character (.) zero or more times (*), which means it captures everything that comes after the second colon.

### What does it do?
This regex is looking for a string that:

1. Starts with the text "Action:".
2. Has a word right after it, separated by a colon.
3. Then, after another colon, it captures everything that follow

In [24]:
import re
action_re = re.compile('^Action: (\w+): (.*)')

In [25]:
# Sample strings
text1 = "Action: Move: North"
text2 = "Action: Jump: High"
text3 = "Error: Not an Action"

In [26]:
# Testing the pattern
match1 = action_re.match(text1)

In [27]:
match1

<re.Match object; span=(0, 19), match='Action: Move: North'>

In [28]:
match1.group(1)

'Move'

In [30]:
match1.group(2)

'North'

In [31]:
match3 = action_re.match(text3)

In [33]:
match3

In [32]:
match3.group(1)

AttributeError: 'NoneType' object has no attribute 'group'

In [59]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [60]:
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [61]:
def calculate(number):
    return eval(number)

In [19]:
calculate("2+2")

4

In [20]:
calculate("20+2*5")

30

In [62]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [63]:
def query(question,max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result
    

In [38]:
prompt

'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop your output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\n\nYour available actions are:\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point\nsyntax if necessary\n\nwikipedia:\ne.g. wikipedia: Django\nReturns a summary from searching Wikipedia\n\nsimon_blog_search:\ne.g. simon_blog_search: Python\nSearch Simon\'s blog for that term\n\nExample session:\nQuestion: What is the capital of France?\nThought: I should look up France on Wikipedia\nAction: wikipedia: France\nPAUSE\n\nYou will be called again with this:\nObservation: France is a country. The capital is Paris.\n\nYou then output:\nAnswer: The capital of France is Paris\n\nPlease Note: if you get 

In [39]:
bot = Chatbot(prompt)

In [41]:
bot("hi how are you?")

"Okay, I'm ready!  \n"

In [43]:
bot("what is your job?")

'Thought: I need to describe what I do to answer that.  I process information and respond to questions.\nAction: simon_blog_search: job\nPAUSE \n'

In [44]:
next_prompt="Tell me about quantum computing from Wikipedia."

In [49]:
next_prompt="Tell me about the current gdp of india from Wikipedia."

In [50]:
result=bot(next_prompt)

In [53]:
result

"Thought: I'll look up India's current GDP on Wikipedia. \nAction: wikipedia: GDP of India\nPAUSE \n"

In [51]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

In [52]:
actions

[<re.Match object; span=(0, 31), match='Action: wikipedia: GDP of India'>]

In [54]:
action, action_input = actions[0].groups()

In [55]:
action

'wikipedia'

In [56]:
action_input

'GDP of India'

In [64]:
query("hi")

Okay, I understand. I'm ready to start the loop and answer your questions!  

Let's begin! 



"Okay, I understand. I'm ready to start the loop and answer your questions!  \n\nLet's begin! \n"

In [65]:
query("how are you?")

Okay, I understand the loop and my available actions.

I am good. 



'Okay, I understand the loop and my available actions.\n\nI am good. \n'

In [66]:
query("Fifteen * twenty five")

Thought: I need to calculate 15 * 25. 

Action: calculate: 15 * 25
PAUSE 

 -- running calculate 15 * 25
Observation: 375
Answer: 375 



'Answer: 375 \n'

In [68]:
query("Has Simon written about AI?")

Thought: I need to check Simon's blog for any mentions of AI. 

Action: simon_blog_search: AI
PAUSE 

 -- running simon_blog_search AI
Observation: It's OK to call it Artificial Intelligence: Update 9th January 2024: This post was clumsily written and failed to make the point I wanted it to make. I've published a follow-up, What I should have said about the term Artificial Intelligence which you should read instead.
My original post follows.


We need to be having high quality conversations about AI: what it can and can't do, its many risks and pitfalls and how to integrate it into society in the most beneficial ways possible.
Any time I write anything that mentions AI it's inevitable that someone will object to the very usage of the term.
Strawman: "Don't call it AI! It's not actually intelligent - it's just spicy autocomplete."
That strawman is right: it's not "intelligent" in the same way that humans are. And "spicy autocomplete" is actually a pretty good analogy for how a lot of th

'Thought: Simon has written about AI and acknowledges the debate surrounding the term.\n\nAnswer: Yes, Simon has written about AI. \n'

In [69]:
print('Thought: Simon has written about AI and acknowledges the debate surrounding the term.\n\nAnswer: Yes, Simon has written about AI. \n')

Thought: Simon has written about AI and acknowledges the debate surrounding the term.

Answer: Yes, Simon has written about AI. 



In [71]:
query("can you tell me capital of usa?")

Thought: I should look up the capital of the USA on Wikipedia. 

Action: wikipedia: Capital of the United States 

PAUSE 

 -- running wikipedia Capital of the United States 
Observation: This is a list <span class="searchmatch">of</span> <span class="searchmatch">capital</span> cities <span class="searchmatch">of</span> <span class="searchmatch">the</span> <span class="searchmatch">United</span> <span class="searchmatch">States</span>, including places that serve or have served as federal, state, insular area, territorial, colonial
Thought: That's not very helpful. I need to be more specific.

Action: wikipedia: Washington D.C. 

PAUSE 

 -- running wikipedia Washington D.C. 
Observation: <span class="searchmatch">Washington</span>, <span class="searchmatch">D.C</span>., formally the District of Columbia and commonly known as <span class="searchmatch">Washington</span> or <span class="searchmatch">D.C</span>., is the capital city and federal district of the United States
Answer: The c

'Answer: The capital of the USA is Washington D.C. \n'